In [0]:
import numpy as np
from scipy.misc import imread, imresize
from glob import glob # helps find files in a folder
import os  # to navigate through filesystem
from google.colab import drive # needed to connect to drive
import matplotlib.pyplot as plt # need to plot images
import os, sys
import glob
from scipy.misc import imread, imresize, imshow
from tflearn.layers.core import input_data
from tflearn.layers.conv import global_avg_pool
from tflearn.layers.core import fully_connected
from tflearn.layers.estimator import regression

In [2]:
drive.mount('/content/drive') # mounts drive to colaboratory

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# navigate through filesystem to class folder
os.chdir('drive/My Drive/DeepLearningFall2018/')

In [8]:
os.listdir() # list all files and folders in this location

['ngrok',
 'DeepLearningFall2018Pretest.gdoc',
 'DeepLearning_Fall2018_HW1.ipynb',
 'Linear Algebra Overview.gslides',
 'turing.jpeg',
 'Basic_Operations.ipynb',
 'Python Numpy_Intro.ipynb',
 'perceptron_image.png',
 'Perceptrons.ipynb',
 'DeepLearning_Fall2018_HW1_answerkey.ipynb',
 'linear_data.csv',
 'DIS_Schedule.gsheet',
 'Homework2_answers.ipynb',
 'Convolutional Neural Networks Lecture 2.gslides',
 'cifar-10-batches-py',
 'DeepLearningFall2018_schedule.gsheet',
 'Colab_Intro.ipynb',
 'iris_flower_data.csv',
 "Midterm_Practice2_teacher's edition.ipynb",
 'Midterm_Practice2.ipynb',
 'Midterm_Practice.ipynb',
 'Midterm_Practice_TeachersEdition.ipynb',
 'Midterm_Exam.ipynb',
 'Midterm_Exam_Answers.ipynb',
 'Grades.gsheet',
 'mnist',
 'ngrok-stable-linux-amd64.zip',
 'Multilayer_perceptron_MichaelTeti.ipynb',
 'Evolution of CNNs.gslides',
 'Data',
 '__pycache__',
 'residual_block.py',
 'residual_block.ipynb',
 'CNN_MichaelTeti.ipynb',
 'Load_Data_Example.ipynb']

In [0]:
from residual_block import *  # import everything from the residual_block notebook

In [0]:
def img_reader(dir, imsz, file_ex=None, sort=False):
    ''' A function that reads in images contained
        in each folder in dir, resizes them to
        imsz[0] x imsz[1], and attaches a label to
        each image based on the folder it came from.

        Args:
             dir: directory where the image folders are
                  located.
             imsz: a list of length 2, where each number
                   represents the height and width each
                   image will be resized to. If all images
                   are the same size and don't need to be
                   resized, just put the size they already
                   are.
              file_ex: if the images you want to compile
                       are all of the same extension and/or
                       there are other file types in the
                       folders that you don't want to read,
                       you can include the extension as file_ex.
                       Example, file_ex = '.jpg'
    '''
    
    main_dir = os.getcwd()
    
    if file_ex is None:
        file_ex = '*'
    else:
        file_ex = '*' + file_ex

    folders = os.listdir(dir)
    os.chdir(dir)
    num_ims=sum([len(files) for r, d, files in os.walk(os.getcwd())])
    imgs = np.zeros([num_ims, imsz[0], imsz[1], 3])
    labels = np.zeros([num_ims, 1])

    for subdir, folder in enumerate(folders):
        os.chdir(dir + '/' + folder)
        files = glob.glob(file_ex)
        if sort:
            files.sort()

        for i, file in enumerate(files):
            if os.path.isfile(file):
                img = imread(file)
                img = imresize(img, [imsz[0], imsz[1]])
                imgs[i, ...] = img
                labels[i, 0] = subdir

    os.chdir(main_dir)            
    return imgs, labels

In [11]:
os.getcwd() # see what folder you are in

'/content/drive/My Drive/DeepLearningFall2018'

In [0]:
os.chdir('Data') # go into Data folder where my two folders are with the images

In [13]:
X, Y = img_reader(os.getcwd(), [150, 150])  # get images and labels as X and Y 

/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


In [0]:
# make labels into vectors with two categories since I have two classes in my dataset
from tflearn.data_utils import to_categorical
Y = to_categorical(Y, 2)

In [14]:
print(X.shape)

(196, 150, 150, 3)


In [0]:
tf.reset_default_graph()

In [0]:
# create the input layer for the network
input_layer = input_data([None, 150, 150, 3])

# first convolution --- 75 x 75 x 64
conv1 = conv_2d(input_layer, 64, 7, strides=2, activation='relu')

# max pooling --- 37 x 37 x 64
pool1 = max_pool_2d(conv1, 3, 2)

# residual blocks 1-3 --- 37 x 37 x 64
residual_block1 = residual_block(pool1, # input to this layer
                                 64, # number of features to look for
                                 3, # filter size
                                 3) # number of residual blocks

# residual blocks 4-7 --- 18 x 18 x 128
residual_block2 = residual_block(residual_block1, # input 
                                 128, # num. features to look for
                                 3, # filter size
                                 4, # number of blocks
                                 2) # stride length 

# residual blocks 8-13 --- 9 x 9 x 256
residual_block3 = residual_block(residual_block2,
                                 256,
                                 3,
                                 6, 
                                 2)

# residual blocks 14-16 --- 4 x 4 x 512
residual_block4 = residual_block(residual_block3,
                                 512,
                                 3,
                                 3,
                                 2)

# global average pooling --- 512
gap = global_avg_pool(residual_block4)

# output layer
output_layer = fully_connected(gap, 2, activation='softmax')

In [0]:
# describe optimization 
optimizer = regression(output_layer, optimizer='adam', learning_rate=0.001)

In [0]:
# build the model
model = tflearn.DNN(optimizer, tensorboard_verbose=3)

In [0]:
# training
model.fit(X,
          Y,
          batch_size=32,
          n_epoch=100,
          validation_set=0.2,
          show_metric=True,
          snapshot_step=100)